# Scraping

Le scraping consiste à extraire des données de pages Web de manière automatique.

La méthode la plus simple consiste à demander le code HTML de la page désirée et à utiliser sa structure pour accéder aux données intéressantes. Ces données peuvent contenir des liens vers d’autres pages à scraper.

In [4]:
# Ce module permet de faire des «requêtes»: donne-moi telle page Web
import requests

# Ce module sert à parcourir la structure d ela page
from bs4 import BeautifulSoup

In [8]:
# Ici, on envoie une requête pour la page
response = requests.get('http://www.paulronga.ch/', verify=False)

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'paulronga.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
# Quel est le statut de la réponse? Si c’est un 404, la page n’a pas été trouvée
response

<Response [200]>

In [11]:
# On charge la page dans BeautifulSoup
doc = BeautifulSoup(response.text)
doc

<!DOCTYPE html>
<html lang="fr-FR">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://paulronga.ch/xmlrpc.php" rel="pingback"/>
<title>Paul Ronga – page personnelle</title>
<meta content="max-image-preview:large" name="robots"/>
<link href="//fonts.googleapis.com" rel="dns-prefetch"/>
<link href="//s.w.org" rel="dns-prefetch"/>
<link href="https://paulronga.ch/feed/" rel="alternate" title="Paul Ronga » Flux" type="application/rss+xml"/>
<link href="https://paulronga.ch/comments/feed/" rel="alternate" title="Paul Ronga » Flux des commentaires" type="application/rss+xml"/>
<script type="text/javascript">
			window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/13.0.1\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/13.0.1\/svg\/","svgExt":".svg","source":{"concatemoji":"https:\/\/paulronga.ch\/wp-includes\/js\/wp-emoji-

## Parcourir la structure de la page
A partir de cette étape, la structure de la page est contenue dans notre variable `doc`.

HTML est structuré par des balises, des mots-clés entourés des signes < et >:

* un paragraphe est signalé par les balises `<p>` (au début) et `</p>` (à la fin)
* le titre principal est entre les balises `<h1>` et `</h1>` (header 1)
* un tableau est entre les balises `<table>` … `</table>`
* une ligne de tableau est entre les balises `<tr>` … `</tr>` (table row)
* une cellule de tableau est entre les balises `<td>` … `</td>` (table data)

Nous pouvons demander à BeautifulSoup de nous chercher des balises HTML avec les fonctions:

`find`: trouver la première balise correspondante

`find_all`: trouver toutes les balises correspondantes

In [5]:
link = doc.find('a')
link

<a class="skip-link screen-reader-text" href="#content">Aller au contenu</a>

In [12]:
links = doc.find_all('a')

# Cela m’a donné 32 liens!
len(links)

links

[<a class="skip-link screen-reader-text" href="#content">Aller au contenu</a>,
 <a href="https://paulronga.ch/" rel="home">Paul Ronga</a>,
 <a href="#" id="nav-toggle">Menu<span></span></a>,
 <a aria-current="page" href="http://paulronga.ch">Accueil</a>,
 <a aria-current="page" href="http://paulronga.ch/#about">Le journalisme data</a>,
 <a aria-current="page" href="http://paulronga.ch/#video">Vidéo</a>,
 <a aria-current="page" href="http://paulronga.ch/#portfolio">Portfolio</a>,
 <a aria-current="page" href="http://paulronga.ch/#news">Blog</a>,
 <a class="btn btn-lg btn-theme-primary" href="#features">Démarrer</a>,
 <a class="btn btn-lg btn-secondary-outline" href="#contact">Contact</a>,
 <a href="http://www.holovaty.com/writing/fundamental-change/" target="_blank">A fundamental way newspaper sites need to change</a>,
 <a href="https://paulronga.ch/les-grandes-gueules-du-grand-conseil/">Les grandes gueules du Grand Conseil</a>,
 <a class="service-button" href="https://paulronga.ch/les-

In [7]:
p_list = doc.find_all('p')
len(p_list)

18

## A vous

Essayez d’autres balises, par exemple:

* doc.find_all('p')
* doc.find_all('img')
* doc.find_all('span')

In [15]:
paragraphs = doc.find_all("p")
len(paragraphs)

18

In [18]:
images = doc.find_all("img")
len(images)


3

In [20]:
number_of_spans = len(doc.find_all("span"))
number_of_spans

9

## Trouver les attributs
Trouver des liens, c’est bien. Mais comment extraire l’URL vers lequel ils conduisent?

La méthode `.get()` est faite pour ça: elle récupère les attributs d’une balise HTML, comme l’attribut «href» qui contient ces URL.

In [22]:
links[6].get('href')
links[6]

<a aria-current="page" href="http://paulronga.ch/#portfolio">Portfolio</a>

## … et avec une boucle
Mettons que nous voulons récupérer toutes les URL de notre menu:
* avec une boucle `for`, on peut passer les liens en revue
* la méthode `.get()` nous permettra de récupérer l’URL de chaque lien

In [9]:
for a in doc.select('.nav-menu a'):
    print(a.get('href'))

http://paulronga.ch
http://paulronga.ch/#about
http://paulronga.ch/#video
http://paulronga.ch/#portfolio
http://paulronga.ch/#news


## Selecteurs
Les balises html contiennent parfois les attributs **id** et/ou **class**. Concrètement, ça se présente par exemple comme ça:

`<table id="resultats" class="bordered striped">`

L’attribut **id** (identifiant) est unique: un seul élément le porte.

L’attribut **class** (classes) peut être ajouté à plusieurs éléments.

Les classes et identifiants servent à styliser la page: tels éléments ont une bordure rouge, un fond gris clair, etc. Ces instructions sont regroupées dans du [code CSS](https://www.w3.org/Style/Examples/011/firstcss.fr.html), un langage informatique qui définit la présentation du HTML.

Dans le code CSS, on les éléments du HTML par des **sélecteurs**. En voici des exemples:

`table
#resultats
table.striped
.bordered`

Tous ces sélecteurs renvoient à la `<table id="resultats" class="bordered striped">` donnée comme exemple plus haut. On peut donc cibler une balise HTML par son nom, par son id (en le précédent d’un "#") ou par une de ses classes.

Pour trouver le sélecteur d’un élément qui vous intéresse, vous pouvez inspecter la page (cmd-alt-i dans Firefox ou Chrome), le repérer dans «Elements» grâce à la flèche tout à gauche, puis faire clic droit -> Copy -> CSS selector.

Avec BeautifulSoup, on peut retrouver les éléments correspondants à un sélecteur grâce à la méthode:

`.select('selecteur')`

Cette méthode renvoie une liste, voici un exemple:

In [10]:
doc.select('.nav-menu a')

[<a aria-current="page" href="http://paulronga.ch">Accueil</a>,
 <a aria-current="page" href="http://paulronga.ch/#about">Le journalisme data</a>,
 <a aria-current="page" href="http://paulronga.ch/#video">Vidéo</a>,
 <a aria-current="page" href="http://paulronga.ch/#portfolio">Portfolio</a>,
 <a aria-current="page" href="http://paulronga.ch/#news">Blog</a>]

# Attributs et méthodes

Mettez doc.find('p') dans la variable `test`, puis écrivez `test.` et appuyez sur tab en laissant votre curseur après le point. Quels attributs et méthodes apparaissent? Essayez-en quelques-uns.

In [33]:
test = doc.find("p")
test.string
test.text
identificateur = test.get("id")
print(identificateur)

None


Vous avez trouvez? Allez, je vous en donne trois:
* `.string` donne le texte de la balise uniquement
* `.text` donne le texte de la balise et de toutes les balises qu’elle contient
* `.get()` permet d’obtenir un attribut précis, comme id, class, href

Pour avoir des exemples clairs, on va charger une page spécialement préparée pour votre scraping.

In [36]:
response = requests.get('https://exemple.tcch.ch/scraping/', verify=False)
doc = BeautifulSoup(response.content, 'html.parser')

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'exemple.tcch.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [12]:
# premier paragraphe
p = doc.find('p')
print('La balise et son contenu:', p)
print('Le contenu grâce à .string:', p.string)
print('Le contenu grâce à .text:', p.text)

La balise et son contenu: <p>Ce paragraphe nous intéresse. C’est le premier.</p>
Le contenu grâce à .string: Ce paragraphe nous intéresse. C’est le premier.
Le contenu grâce à .text: Ce paragraphe nous intéresse. C’est le premier.


Quelle différence entre `.string` et `.text`? On le comprend quand notre balise contient d’autres balises. C’est le cas du dernier paragraphe: il contient une balise `<em>`.

In [13]:
# dernier paragraphe
p = doc.select('p')[-1]
p

<p>Le troisième est un peu plus <em>compliqué</em>…<br/>
    Mais on va s’en sortir!</p>

In [14]:
print('.string:', p.string)
print('.text:', p.text)

.string: None
.text: Le troisième est un peu plus compliqué…
    Mais on va s’en sortir!


Quand une balise contient d’autres balises, `.string` renvoie donc **None**, tandis que `.text` renvoie tout le texte contenu dans la balise principale et les balises qu’elle contient.

## A vous
Qu’est-ce que `.string` et `.text` donnent comme résultat avec la table? Essayez!

In [54]:
table = doc.find("table")
print(table)

<table>
<!-- tr = table row -->
<tr>
<th>Chats</th><th>Chiens</th>
</tr>
<tr>
<td>3</td><td>0</td>
</tr>
<tr>
<td>5</td><td>0</td>
</tr>
<tr>
<td>3</td><td>5</td>
</tr>
<tr>
<td>3</td><td>90</td>
</tr>
<tr>
<td>13</td><td>10</td>
</tr>
</table>


In [55]:
print(table.string)

None


In [50]:
print(table.text)




ChatsChiens


30


50


35


390


1310




Le système que nous avons vu ne fonctionne pas pour certaines pages Web construites dynamiquement avec du JavaScript. Dans ce cas, on peut automatiser un navigateur tel que Chrome ou Firefox avec des outils comme [Selenium](https://pypi.org/project/selenium/). C’est nettement plus complexe que d’utiliser requests et BeautifulSoup.